In [1]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *


score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [5]:
def extend_alignment_chaos_multiple(M,str1_nr,A,index_dict,MSA_of_str2, verbose=False): #needs inclusion of str1_nr, to come from the outside.... str1_nr is the name "predecessor" string  
    MA = []
    i = 0
    j = 0
    col_in_M_of_parent_string=int(index_dict[str1_nr]) #getting the column from the index dict that holds the "predecessor" string, to know where it is in the MSA
    while i < len(M) and j < len(A):
        if verbose: print("i:"+str(i)+", j:"+str(j))
        if verbose: print("parent string nr: "+ str(str1_nr))
        if verbose: print("which is in col:"+str(col_in_M_of_parent_string))
        if verbose: print("I compare "+ str(M[i][col_in_M_of_parent_string])+" and "+str(A[j][0]))
        # Case 1:
        if M[i][col_in_M_of_parent_string] == '-' and A[j][0] == '-':
            if verbose: print("I go to case 1 (two gaps)")
            #M[i].append(A[j][1])
            M[i].append(MSA_of_str2[j])#just append the whole damn column!
            MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
            j = j + 1

        # Case 2
        elif M[i][col_in_M_of_parent_string] == '-' and A[j][0] != '-':
            if verbose:print("I go to case 2 (gap,char)")
            for char in MSA_of_str2[0]:#append as many gaps as you there are chars (cols) in MSA of str2
                M[i].append('-')
                MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
        
        # Case 3:
        elif M[i][col_in_M_of_parent_string] != '-' and A[j][0] == '-':
            if verbose:print("I go to case 3 (char, gap)")
            c = ['-']*len(M[i])
            c.append(MSA_of_str2[j])
            MA.append(c)
            if verbose: print("Now MA is this: \n "+ str(MA))
            j = j + 1
        
        # Case 4:
        elif M[i][col_in_M_of_parent_string] != '-' and A[j][0] != '-':
            if verbose: 
                print("I go to case 4 (char, char)")
            M[i].append(MSA_of_str2[j]) #just put the whole damn col/row from the MSA of str2
            MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
            j = j + 1
    #when one of the strings has ended, put in gaps, till the other ends as well. 
    if i < len(M):
        while i < len(M):
            for char in MSA_of_str2[0]:
                M[i].append('-')
                MA.append(M[i])
            i = i + 1
            
    if j < len(A):
        k = len(M[col_in_M_of_parent_string])
        while j < len(A):
            c = ['-']*(k-1)
            c.append(MSA_of_str2[j])
            MA.append(c)
            j = j + 1
    return MA


In [6]:
test_res_mat=np.array([['*', '1', '0', '1'], ['*', '2', '0', '2'],['','3','0','2'], ['', '3', '1', '2']])

In [7]:
def orchestrate_merge(res_matrix, seqs,score_matrix,gapcost,verbose=False): #should also include seqs,score_matrix,gap_cost, verbose=False)
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    print(in_which_MSA_is_it)
    for row in res_matrix:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost,verbose=verbose)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)] #the 'guide' (pairwise alignment) in this iteration!
            #where are these two sequences now? get the MSA and their position in that MSA!
            which_MSA_node1=in_which_MSA_is_it[node1][0]
            which_pos_in_MSA_node1=in_which_MSA_is_it[node1][1]
            which_MSA_node2=in_which_MSA_is_it[node2][0]
            which_pos_in_MSA_node2=in_which_MSA_is_it[node2][1]
            current_MSA_node1=MSA_list[which_MSA_node1][which_pos_in_MSA_node1] #how is the current MSA of node 1 looking??
            current_MSA_node2=MSA_list[which_MSA_node2][which_pos_in_MSA_node2] #how is the current MSA of node 1 looking??
            #now we should compare the A (pairwise alignment) to the current alignment
         





#merge_actually() #TODO: #um actually make this lollll




In [8]:
names=np.unique(test_res_mat[:, 2:])
print(names)
#in_which_MSA_is_itt = {name:[int(name):0 for i, name in enumerate(names)]}
in_which_MSA_is_itt = {name: [int(name),0] for name in names}
print(in_which_MSA_is_itt)

seqss=['agct','ctgg','gtacccacaca']
MSA_list=[[*seq] for seq in seqss]
print(MSA_list)


['0' '1' '2']
{'0': [0, 0], '1': [1, 0], '2': [2, 0]}
[['a', 'g', 'c', 't'], ['c', 't', 'g', 'g'], ['g', 't', 'a', 'c', 'c', 'c', 'a', 'c', 'a', 'c', 'a']]


In [2]:
def make_MSA_position_overview_dict(res_matrix):#initializes the overview dictionary
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    print(in_which_MSA_is_it)

#make_MSA_position_overview_dict(test_res_mat)

In [3]:
def find_highest_second_element(d, first_element): #to find the highest col nr already in the MSA
    highest_second_element = 0
    for key, value in d.items():
        if value[0] == first_element:
            second_element = value[1]
            if second_element > highest_second_element:
                highest_second_element = second_element
    return highest_second_element

In [4]:
def update_MSA_position_overview_dict(MSA_position_overview_dict,node1,node2): #may be somehow inadequate!
    print(node1,node2)
    node1_origin_MSA=MSA_position_overview_dict[node1][0]
    node2_origin_MSA=MSA_position_overview_dict[node2][0]
    node1_origin_MSA_col=MSA_position_overview_dict[node1][1]
    node2_origin_col=MSA_position_overview_dict[node2][1]
    #now I have the original positions in from the position overview dict!
    nr_we_got_to=find_highest_second_element(MSA_position_overview_dict,MSA_position_overview_dict[node1])
    MSA_position_overview_dict[node2]=[node1_origin_MSA,(nr_we_got_to+1)] #update the pos_dict to show that node2 changed MSA and make the col one bigger than the max
    return MSA_position_overview_dict

overview={'0': [0, 0], '1': [1, 0], '2': [2, 0]}
update_MSA_position_overview_dict(overview,'1','2')

1 2


{'0': [0, 0], '1': [1, 0], '2': [1, 1]}

In [5]:
def get_MSA_pos(MSA_position_overview_dict, node):
    value=MSA_position_overview_dict[node]
    MSA_pos=value[0]
    MSA_pos_col=value[1]
    return MSA_pos,MSA_pos_col

get_MSA_pos(overview,'2')[0]

1

In [30]:
def identify_merge_case(guide_PA,MSA1,MSA2,node1,node2,MSA_position_overview_dict,i,j,l,m,orig_str_node2): 
    case=None
    #node1_col=get_MSA_pos(MSA_position_overview_dict,node1)[1]
    node1_position=MSA_position_overview_dict[node1]
    node1_col=node1_position[1]
    node2_position=MSA_position_overview_dict[node2]
    node2_col=node2_position[1]
    print("i and j, and l are: "+str(i)+" and "+str(j)+' and '+str(l))
    if MSA2[l][node2_col]=='-'and orig_str_node2[m]!='-':
        case=0
    elif guide_PA[i][0]=='-' and MSA1[j][int(node1_col)]=='-':
        case=1
    elif guide_PA[i][0]!='-' and MSA1[j][int(node1_col)]=='-':
        case=2
    elif guide_PA[i][0]=='-' and MSA1[j][int(node1_col)]!='-':
        case=3
    elif guide_PA[i][0]!='-' and MSA1[j][int(node1_col)]!='-':
        case=4
    print("Case: "+str(case))
    return case

fake_MSA1=[['-','a'],['a','g']]
fake_MSA2=[['-','a'],['a','g','c']]
fake_guide_PA=[['-','c'],['-','g']]

#identify_merge_case(fake_guide_PA,fake_MSA1,fake_MSA2,'0','1',overview,1,1,1) #sounds about right!!



In [29]:
def handle_case0(MSA1,MSA2,i,j,l):
    #col_new=list(MSA1[j])+list(MSA2[l])#I think this is wrong
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])

    return new_col
#ok we have the guide_PA, and the MSA1 and MSA2 that the strings are currently in, the MSA_position_overview to keep track of
#in which col a node/string is in. We use this func when guide_PA[i][0]=='-' and MSA1[j][node1_col]=='-'.if they are both at a gap.
#so this is a case when we just wanna put the whole thing of MSA (of that colum), because it looks good!
#assuming that the guide_PA was made to between how the strings were already looking in the MSA's it the columns in the guide PA and
#MSA would be the same enableling a simple enough merge. May be a stupid choice, but let's see.
def handle_case1(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new
#this next case is invoked if guide_PA[i][0]!='-' and MSA1[j][node1_col]=='-'.
#so the guide wants a non-gap. But we are at a gap in the MSA1. So we need to add gaps for every row in the col of MSA2, and put none of MSA2 in there.
def handle_case2(MSA1,MSA2,i,j,l):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_append=['-']*how_many_cols_in_MSA2
    col_now=MSA1[j]
    col_new=list(MSA1[j])+gaps_to_append
    return col_new

#now we are in a situation where guide_PA[i][0]=='-' and MSA1[j][node1_col]!='-'.
#meaning that the guide wants a gap, and but there is a char in the MSA.
#so now we need to add amount of gaps corresponding to the rows in a col of MSA1. And then add the whole col of MSA1.
def handle_case3(MSA1,MSA2,i,j,l): #this case seems solid!
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])
    return new_col

#now we are in a situation where guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-'.
#meaning that we are in a place where we just need to put in the whole col from MSA2 into MSA1
def handle_case4(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new


fake_MSA1=[['a','a','g'],['a','g','t']]
fake_MSA2=[['c','-','a'],['-','a','a']]
fake_guide_PA=[['a','c'],['-','g']]
overview={'0': [0, 0], '1': [1, 0], '2': [2, 0]}
#case=identify_merge_case(fake_guide_PA,fake_MSA1,fake_MSA2,'0','1',overview,0,0,0,)
#print(case)
#result=handle_case4(fake_MSA1,fake_MSA2,0,0,0)
#print(result)



In [6]:
#to be used if we have reached the end of the guide MSA, but not the MSA1
def finish_running_through_MSA1(MSA1,MSA2,j,i,l):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_add=['-']*how_many_cols_in_MSA2
    new_col=list(MSA1[j])+gaps_to_add
    return new_col

def finish_running_through_MSA2(MSA1,MSA2,j,i,l):
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])
    return new_col

fake_MSA1=[['a','a','g'],['a','g','t'],['a','t','c']]
fake_MSA2=[['c','-','a'],['-','a','a']]

res1=finish_running_through_MSA1(fake_MSA1,fake_MSA2,2,1,1)
res2=finish_running_through_MSA2(fake_MSA1,fake_MSA2,2,1,1)

print(res1,res2)

['a', 't', 'c', '-', '-', '-'] ['-', '-', '-', '-', 'a', 'a']


In [39]:
#this function is to to ONE merge of two strings and their respective MSA that they are stuck in...
#I think this function assumes that the col/row position of the an element is the same in MSA2 and in guide_PA #FIXME:#could be a prob.
def merge_united(guide_PA,MSA_list,in_which_MSA_is_it,node1,node2,orig_str_node2):
    print(type(in_which_MSA_is_it))
    print(in_which_MSA_is_it)
    print(type(node1))
    print(type(node2))
    MSA_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)[0]
    MSA_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)[0]
    print(MSA_of_node1)
    print(MSA_of_node2)
    #print(test_ahah)
    #MSA_of_node1,row_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)
    #MSA_of_node2,row_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)
    MSA1=MSA_list[int(MSA_of_node1)]
    MSA2=MSA_list[int(MSA_of_node2)]
    
    j=0
    i=0
    l=0
    m=0
    MSA_new=[]
    while j < len(MSA1) and i < len(guide_PA) and l<len(MSA2): #FIXME:#is this even the correct while-statement????  
        case=identify_merge_case(guide_PA,MSA1,MSA2,node1,node2,in_which_MSA_is_it,j,i,l,m,orig_str_node2)
        if case==0:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            handle_case0(MSA1,MSA2,i,j,l)
            l+=1
            m+=1
        elif case==1:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=handle_case1(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
            i+=1
            l+=1
            m+=1
        elif case==2:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=handle_case2(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
        elif case==3:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=handle_case3(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            i+=1
            l+=1
            m+=1
        elif case==4:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=handle_case4(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
            i+=1
            l+=1
            m+=1
        else:
            print("Ooops, you hit no case, gotta go!")
            exit
    if j < len(MSA1):
        while j< len(MSA1):
            column=finish_running_through_MSA1(MSA1,MSA2,j,i,l)
            j+=1
            #l+=1
            MSA_new.append(column)
    if l< len(MSA2):
        while l< len(MSA2):
            column=finish_running_through_MSA2(MSA1,MSA2,j,i,l)
            l+=1
            MSA_new.append(column)
    return MSA_new #FIXME:#also need to update the postion dictionary and possible other things...

        

In [2]:

fake_MSA1=[['a','a','g'],['a','g','t']]
fake_MSA2=[['c','c','a'],['c','a','a']]
fake_MSA_list=[fake_MSA1,fake_MSA2]
print(fake_MSA_list)
fake_guide_PA=[['a',''],['-','a']]
overview={'0': [0, 0], '1': [1, 0], '2': [2, 0]}


merge_united(fake_guide_PA,fake_MSA_list,overview,'0','1')

[[['a', 'a', 'g'], ['a', 'g', 't']], [['c', 'c', 'a'], ['c', 'a', 'a']]]


TypeError: merge_united() missing 1 required positional argument: 'orig_str_node2'

In [23]:
def equality_here_test(a,b):
    equality=False
    if a=='-' and b=='-' or a!='-' and b!='-':
        equality=True
    return equality
def alt_indentify_case(guide_PA,MSA1,MSA2,node1,node2,MSA_position_overview_dict,i,j,l):
    case=None
    try:
        node1_position=MSA_position_overview_dict[node1]
        node1_col=node1_position[1]
        node2_position=MSA_position_overview_dict[node2]
        node2_col=node2_position[1]
        #if equality_here_test(guide_PA[i][0], MSA1[j][int(node1_col)])== True and equality_here_test(guide_PA[i][1],MSA2[l][int(node2_col)])==True:case=3
        if (guide_PA[i][0] == MSA1[j][int(node1_col)] and guide_PA[i][1] == MSA2[l][int(node2_col)]) or \
        (guide_PA[i][0] != '-' and MSA1[j][int(node1_col)] != '-' and guide_PA[i][1] != '-' and MSA2[l][int(node2_col)] != '-'):
           case = 3
        elif equality_here_test(MSA1[j][int(node1_col)],guide_PA[i][0])==False and equality_here_test(MSA2[l][int(node2_col)],guide_PA[i][1])==False: case=4
        elif guide_PA[i][0]=='-' and MSA1[j][int(node1_col)]=='-': case=1
        elif guide_PA[i][0]!='-' and MSA1[j][int(node1_col)]!='-' and MSA2[l][int(node2_col)]=='-': case=1
        elif guide_PA[i][0]!='-' and MSA1[j][int(node1_col)]!='-' and guide_PA[i][1]=='-':case=2
        elif guide_PA[i][1]=='-' and MSA2[l][int(node2_col)]=='-': case=2
        elif guide_PA[i][1]!='-' and MSA2[l][int(node2_col)]!='-' and MSA1[j][int(node1_col)]=='-':case=2
        elif guide_PA[i][1]!='-' and MSA2[l][int(node2_col)]!='-' and guide_PA[i][0]=='-': case=1
        else:
            print("Ooops, you hit no case, gotta go!")
            case='STOP!!!'
        print("case:"+str(case))
        return case
    except IndexError as e:
        print("An IndexError occurred:", e)
        sys.exit()

#1 is guide_PA[i][0]
#2 is guide_PA[i][1]
#3 is MSA1[j][int(node1_col)]
#4 is MSA2[l][int(node2_col)]

def alt_handle_case1(MSA1,MSA2,i,j,l):
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])
    return new_col #after this, increase only l
def alt_handle_case2(MSA1,MSA2,i,j,l):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_append=['-']*how_many_cols_in_MSA2
    col_now=MSA1[j]
    col_new=list(MSA1[j])+gaps_to_append
    return col_new #after this, increase only j
def alt_handle_case3(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new #after this, increase i, j and l
def alt_handle_case4(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new #after this, increase j and l, but not i
    
def alt_merge_united(guide_PA,MSA_list,in_which_MSA_is_it,node1,node2):
    print(type(in_which_MSA_is_it))
    print(in_which_MSA_is_it)
    print(type(node1))
    print(type(node2))
    MSA_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)[0]
    MSA_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)[0]
    spot_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)[1]
    spot_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)[1]
    print(MSA_of_node1)
    print(MSA_of_node2)
    #print(test_ahah)
    #MSA_of_node1,row_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)
    #MSA_of_node2,row_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)
    MSA1=MSA_list[int(MSA_of_node1)]
    MSA2=MSA_list[int(MSA_of_node2)]
    
    j=0
    i=0
    l=0
    MSA_new=[]
    while j <= (len(MSA1)-1) and i <= (len(guide_PA)-1) and l<=(len(MSA2)-1): # FIXME: #um the print statement may be outdated...
        print("I don't have the case identified yet, but MSA1 is: " + str(MSA1[j]) + " MSA2 is: " + str(MSA2[i]) + " ,and the guide is: " + str(guide_PA[i]))
        print("nodes pointing to cols: " + str(spot_of_node1) + " , " + str(spot_of_node2))
        print("i,j,l: "+str(i)+','+str(j)+','+str(l))
        print("max for those should be: "+str(len(guide_PA))+','+str(len(MSA1))+','+str(len(MSA2)))

        case=alt_indentify_case(guide_PA,MSA1,MSA2,node1,node2,in_which_MSA_is_it,j,i,l)
        if case=='STOP!!!':
            sys.exit()
        if case==1:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=alt_handle_case1(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            l+=1
        elif case==2:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=alt_handle_case2(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
        elif case==3:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=alt_handle_case3(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
            i+=1
            l+=1
        elif case==4:
            print("MSA1 is: "+str(MSA1[j])+"MSA2 is: "+str(MSA2[i])+" ,and the guide is: "+str(guide_PA[i]))
            column=alt_handle_case4(MSA1,MSA2,i,j,l)
            MSA_new.append(column)
            j+=1
            l+=1
    if j <= (len(MSA1)-1):
        while j<= (len(MSA1)-1):
            column=finish_running_through_MSA1(MSA1,MSA2,j,i,l)
            j+=1
            #l+=1
            MSA_new.append(column)
    if l<= (len(MSA2)-1):
        while l<= (len(MSA2)-1):
            column=finish_running_through_MSA2(MSA1,MSA2,j,i,l)
            l+=1
            MSA_new.append(column)
    if i<=(len(guide_PA)-1):
        print('yikes, the strings have run out but the guide has not')
        i+=1
    return MSA_new


def alt_the_whole_thing(seqs,score_matrix,gapcost):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    print("matrix for MST: "+str(matrix_for_MST))
    min_span_edges_res=find_min_span_edges_testing(matrix_for_MST)
    print("min span edges: "+str(min_span_edges_res))
    in_which_MSA_is_it={}
    names= np.unique(min_span_edges_res[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    #MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    MSA_list=[[[char] for char in seq] for seq in seqs] #let's try this instead!
    print("here comes the pos-dict: "+str(in_which_MSA_is_it))
    print("look at the current MSA-list: "+str(MSA_list))
    for row in min_span_edges_res:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            orig_str_node2=seqs[int(node2)]
            united_MSA_new=alt_merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2) #should bring the merged alignment. Just need to update MSA_list and the dict now then...but also for the others that were joined into it now......
            print("this is the union produced: "+ str(united_MSA_new))
            which_spot_in_MSA_list_to_update=in_which_MSA_is_it[node1][0]
            which_spot_in_MSA_list_to_remove=in_which_MSA_is_it[node2][0]
            #in_which_MSA_is_it=update_MSA_position_overview_dict(in_which_MSA_is_it,node1,node2)
            
            MSA_list[which_spot_in_MSA_list_to_update]=united_MSA_new
            MSA_list.pop(which_spot_in_MSA_list_to_remove)
            #now to keep track of where everything is, a nightmare..
            companions_list=[] #to keep the keys of the strings already in MSA2 for later update.
            for key, value in in_which_MSA_is_it.items():
                how_many_cols_already_in_MS1=find_highest_second_element(in_which_MSA_is_it,which_spot_in_MSA_list_to_update)
                if value[0]>which_spot_in_MSA_list_to_remove:
                    value[0]-=1
                elif value[0]==which_spot_in_MSA_list_to_remove:
                    value[0]=which_spot_in_MSA_list_to_update
                    companions_list.append(key)
            print("this is how many cols already in MSA1: "+str(how_many_cols_already_in_MS1))
            print("this is the companions list: "+str(companions_list))
            for element in companions_list:
                col_of_element_in_old_MSA2=in_which_MSA_is_it[element][1]
                print("this is column nr of node1 in old MSA2: "+str(col_of_element_in_old_MSA2))
                col_in_new_MSA_node2=int(how_many_cols_already_in_MS1)+int(col_of_element_in_old_MSA2)+1
                print("new col nr of node2: "+str(col_in_new_MSA_node2))
                print("this is the place I try to change: "+ str(in_which_MSA_is_it[element][1]))
                in_which_MSA_is_it[element][1]=col_in_new_MSA_node2
                print("this is the updated dict: "+str(in_which_MSA_is_it))
        print(MSA_list)
    print(MSA_list)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(total_cost,MSA_list)

            



In [26]:
def the_whole_thing(seqs,score_matrix,gapcost):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    print("matrix for MST: "+str(matrix_for_MST))
    min_span_edges_res=find_min_span_edges_testing(matrix_for_MST)
    print("min span edges: "+str(min_span_edges_res))
    in_which_MSA_is_it={}
    names= np.unique(min_span_edges_res[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    #MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    MSA_list=[[[char] for char in seq] for seq in seqs] #let's try this instead!
    print("here comes the pos-dict: "+str(in_which_MSA_is_it))
    print("look at the current MSA-list: "+str(MSA_list))
    for row in min_span_edges_res:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            orig_str_node2=seqs[int(node2)]
            united_MSA_new=merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2,orig_str_node2) #should bring the merged alignment. Just need to update MSA_list and the dict now then...but also for the others that were joined into it now......
            print("this is the union produced: "+ str(united_MSA_new))
            which_spot_in_MSA_list_to_update=in_which_MSA_is_it[node1][0]
            which_spot_in_MSA_list_to_remove=in_which_MSA_is_it[node2][0]
            #in_which_MSA_is_it=update_MSA_position_overview_dict(in_which_MSA_is_it,node1,node2)
            
            MSA_list[which_spot_in_MSA_list_to_update]=united_MSA_new
            MSA_list.pop(which_spot_in_MSA_list_to_remove)
            #now to keep track of where everything is, a nightmare..
            companions_list=[] #to keep the keys of the strings already in MSA2 for later update.
            for key, value in in_which_MSA_is_it.items():
                how_many_cols_already_in_MS1=find_highest_second_element(in_which_MSA_is_it,which_spot_in_MSA_list_to_update)
                if value[0]>which_spot_in_MSA_list_to_remove:
                    value[0]-=1
                elif value[0]==which_spot_in_MSA_list_to_remove:
                    value[0]=which_spot_in_MSA_list_to_update
                    companions_list.append(key)
            print("this is how many cols already in MSA1: "+str(how_many_cols_already_in_MS1))
            print("this is the companions list: "+str(companions_list))
            for element in companions_list:
                col_of_element_in_old_MSA2=in_which_MSA_is_it[element][1]
                print("this is column nr of node1 in old MSA2: "+str(col_of_element_in_old_MSA2))
                col_in_new_MSA_node2=int(how_many_cols_already_in_MS1)+int(col_of_element_in_old_MSA2)+1
                print("new col nr of node2: "+str(col_in_new_MSA_node2))
                print("this is the place I try to change: "+ str(in_which_MSA_is_it[element][1]))
                in_which_MSA_is_it[element][1]=col_in_new_MSA_node2
                print("this is the updated dict: "+str(in_which_MSA_is_it))
        print(MSA_list)
    print(MSA_list)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(total_cost,MSA_list)

            

In [55]:
def alt_alt_identify_merge_case(guide_PA,MSA1,MSA2,node1,node2,MSA_position_overview_dict,i,j,l):
    node1_position=MSA_position_overview_dict[node1]
    node1_col=node1_position[1]
    node2_position=MSA_position_overview_dict[node2]
    node2_col=node2_position[1]
    case=None
    move_on_in_guide_flag=False
    if equality_here_test(guide_PA[i][0],MSA1[j][node1_col])==True and equality_here_test(guide_PA[i][1],MSA2[l][node2_col])==True:
        case=3
    #elif equality_here_test(guide_PA[i][0],MSA1[j][node1_col])==False and equality_here_test(guide_PA[i][1],MSA2[l][node2_col]==False):
    elif equality_here_test(guide_PA[i][0], MSA1[j][node1_col]) == False and equality_here_test(guide_PA[i][1], MSA2[l][node2_col]) == False:
        case=4
    elif guide_PA[i][0]=='-' and MSA1[j][node1_col]=='-':
        case=1
        if guide_PA[i][0]=='-' or guide_PA[i][1]=='-':
            move_on_in_guide_flag=True
    elif guide_PA[i][1]=='-' and MSA2[l][node2_col]=='-':
        case=2
        if guide_PA[i][0]=='-' or guide_PA[i][1]=='-':
            move_on_in_guide_flag=True
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-' and MSA2[l][node2_col]=='-':
        case=1
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-' and guide_PA[i][1]=='-':
        case=2
    elif guide_PA[i][1]!='-' and MSA2[l][node2_col]!='-' and guide_PA[i][0]=='-':
        case=1
    elif guide_PA[i][1]!='-' and MSA2[l][node2_col]!='-' and MSA1[j][node1_col]=='-':
        case=2
    else:
        sys.exit()
    return case,move_on_in_guide_flag

    
def alt_alt_merge_united(guide_PA,MSA_list,in_which_MSA_is_it,node1,node2):
    print(type(in_which_MSA_is_it))
    print(in_which_MSA_is_it)
    print(type(node1))
    print(type(node2))
    MSA_of_node1=in_which_MSA_is_it[node1][0]
    MSA_of_node2=in_which_MSA_is_it[node2][0]
    spot_of_node1=in_which_MSA_is_it[node1][1]
    spot_of_node2=in_which_MSA_is_it[node2][1]
    print(MSA_of_node1)
    print(MSA_of_node2)
    MSA1=MSA_list[int(MSA_of_node1)]
    MSA2=MSA_list[int(MSA_of_node2)]
    
    j=0 
    i=0
    l=0
    MSA_new=[]
    while i<=(len(guide_PA)-1) and j<=(len(MSA1)-1) and l<=(len(MSA2)-1):
        print("I don't have the case identified yet, but MSA1 is: " + str(MSA1[j]) + " MSA2 is: " + str(MSA2[i]) + " ,and the guide is: " + str(guide_PA[i]))
        print("nodes pointing to cols: " + str(spot_of_node1) + " , " + str(spot_of_node2))
        print("i,j,l: "+str(i)+','+str(j)+','+str(l))
        print("max for those should be: "+str(len(guide_PA))+','+str(len(MSA1))+','+str(len(MSA2)))
        case,move_on_in_guide_flag=alt_alt_identify_merge_case(guide_PA,MSA1,MSA2,node1,node2, in_which_MSA_is_it,i,j,l)
        print("case:"+str(case))
        if case==1:
            col=alt_handle_case1(MSA1,MSA2,i,j,l)
            MSA_new.append(col)          
            l+=1
            if move_on_in_guide_flag==True:
                i+=1
        if case==2:
            col=alt_handle_case2(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            j+=1
            if move_on_in_guide_flag==True:
                i+=1
        if case==3:
            col=alt_handle_case3(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            i+=1
            j+=1
            l+=1
        if case==4:
            col=alt_handle_case4(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            j+=1
            l+=1
    if j <= (len(MSA1)-1):
        while j<= (len(MSA1)-1):
            column=finish_running_through_MSA1(MSA1,MSA2,j,i,l)
            j+=1
            #l+=1
            MSA_new.append(column)
    if l<= (len(MSA2)-1):
        while l<= (len(MSA2)-1):
            column=finish_running_through_MSA2(MSA1,MSA2,j,i,l)
            l+=1
            MSA_new.append(column)
    if i<=(len(guide_PA)-1):
        print('yikes, the strings have run out but the guide has not')
        i+=1
    return MSA_new


def alt_alt_the_whole_thing(seqs,score_matrix,gapcost):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    print("matrix for MST: "+str(matrix_for_MST))
    min_span_edges_res=find_min_span_edges_testing(matrix_for_MST)
    print("min span edges: "+str(min_span_edges_res))
    in_which_MSA_is_it={}
    names= np.unique(min_span_edges_res[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    #MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    MSA_list=[[[char] for char in seq] for seq in seqs] #let's try this instead!
    print("here comes the pos-dict: "+str(in_which_MSA_is_it))
    print("look at the current MSA-list: "+str(MSA_list))
    for row in min_span_edges_res:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            orig_str_node2=seqs[int(node2)]
            united_MSA_new=alt_alt_merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2)#should bring the merged alignment. Just need to update MSA_list and the dict now then...but also for the others that were joined into it now......
            print("this is the union produced: "+ str(united_MSA_new))
            which_spot_in_MSA_list_to_update=in_which_MSA_is_it[node1][0]
            which_spot_in_MSA_list_to_remove=in_which_MSA_is_it[node2][0]
            #in_which_MSA_is_it=update_MSA_position_overview_dict(in_which_MSA_is_it,node1,node2)
            
            MSA_list[which_spot_in_MSA_list_to_update]=united_MSA_new
            MSA_list.pop(which_spot_in_MSA_list_to_remove)
            #now to keep track of where everything is, a nightmare..
            companions_list=[] #to keep the keys of the strings already in MSA2 for later update.
            for key, value in in_which_MSA_is_it.items():
                how_many_cols_already_in_MS1=find_highest_second_element(in_which_MSA_is_it,which_spot_in_MSA_list_to_update)
                if value[0]>which_spot_in_MSA_list_to_remove:
                    value[0]-=1
                elif value[0]==which_spot_in_MSA_list_to_remove:
                    value[0]=which_spot_in_MSA_list_to_update
                    companions_list.append(key)
            print("this is how many cols already in MSA1: "+str(how_many_cols_already_in_MS1))
            print("this is the companions list: "+str(companions_list))
            for element in companions_list:
                col_of_element_in_old_MSA2=in_which_MSA_is_it[element][1]
                print("this is column nr of node1 in old MSA2: "+str(col_of_element_in_old_MSA2))
                col_in_new_MSA_node2=int(how_many_cols_already_in_MS1)+int(col_of_element_in_old_MSA2)+1
                print("new col nr of node2: "+str(col_in_new_MSA_node2))
                print("this is the place I try to change: "+ str(in_which_MSA_is_it[element][1]))
                in_which_MSA_is_it[element][1]=col_in_new_MSA_node2
                print("this is the updated dict: "+str(in_which_MSA_is_it))
        print(MSA_list)
    print(MSA_list)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(total_cost,MSA_list)

            

        

In [56]:
seqs_for_small_test=['cgcgc','gtaa','atacg']
brca_seqs_work_please,names=parse_fasta_multiple('brca1-testseqs.fasta')
gap_cost=5
alt_alt_the_whole_thing(brca_seqs_work_please,score_matrix,gap_cost)


Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]]
Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ 18.   0. 385. 455.  44. 455.  56. 469.]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]]
Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ 18.   0. 385. 455.  44. 455.  56. 469.]
 [384. 385.   0. 440. 385. 455. 391. 354.

(38602,
 [[['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 't', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 't', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 'a', '-', '-'],
   ['-', '-', '-', '-', '-', 'a', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 't', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 'a', '-', '-'],
   ['-', '-', '-', '-', '-', 'a', '-', '-'],
   ['-', '-', '-', '-', '-', 'g', '-', '-'],
   ['-', '-', '-', '-', '-', 'a', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['-', '-', '-', '-', '-', 't', '-', '-'],
  